In [ ]:
!pip install scikit-plot
!pip install --upgrade tables
!pip install xgboost
!pip install hyperopt

### Biblioteki

In [ ]:
import pandas as pd
import numpy as np
import re
np.random.seed(0)
from collections import defaultdict
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.dummy import DummyRegressor
import xgboost as xgb
from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from functools import partial
import scikitplot as skplt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Dane

In [ ]:
train = pd.read_hdf('/content/drive/My Drive/train.car_price.h5')
train = train.copy()
test = pd.read_hdf('/content/drive/My Drive/test.car_price.h5') 
test = test.copy()

In [ ]:
df_all = pd.concat([train, test])

## Potrzebne funkcje

In [ ]:
def num_filter(val):
    return re.sub('[^0-9]','', val)

def float_filter(val):
    return re.sub('[^0-9\,\.]','', val).replace(',', '.')

def str_filter(val):
    return val.lower().strip()

def pol_ang_con(df, col1, col2, col, funkcja=(lambda x: x), value_none=None):
    df[col] = [funkcja(a) if a != None else funkcja(b) if b != None else value_none for a, b in zip(df[col1], df[col2])]
    del df[col1]
    del df[col2]
    return df

months_dict = {'styczeń': 1, 'january': 1, 'luty': 2, 'february': 2, 'marzec': 3, 'march': 3, 'kwiecień': 4, 'april': 4, 'maj': 5, 'may': 5, 'czerwiec': 6, 'june': 6, 'lipiec': 7, 'july': 7, 'sierpień': 8, 'august': 8, 'wrzesień': 9, 'september': 9, 'październik': 10, 'october': 10, 'listopad': 11, 'november': 11, 'grudzień': 12, 'december': 12}
def przyk(x):
  if x != None:
    x = x.split(' ')
    if len(x) == 3:
      day = x[0]
      month = x[1]
      year = x[2]
      month = months_dict[month]
      return '{0}/{1}/{2}'.format(day, month, year)
    else:
      month = x[0]
      year = x[1]
      month = months_dict[month]
      return '{0}/{1}'.format(month, year)

## Połączenie kolumn - polski + angielski

In [ ]:
bool_funk = lambda x: x == 'Tak'
pol_ang_con(df_all, 'param_rok-produkcji', 'param_year', 'param_rok-produkcji_new', funkcja = lambda x: int(x), value_none=-1)
pol_ang_con(df_all, 'param_moc', 'param_engine-power', 'param_moc_new', funkcja = lambda x: int(x.split(' ')[0]), value_none=-1)
pol_ang_con(df_all, 'param_przebieg', 'param_mileage', 'param_przebieg_new', funkcja = lambda x: int(x.split('km')[0].replace(' ','')), value_none=-1)
pol_ang_con(df_all, 'param_metallic', 'param_metalik', 'param_metalik_new', funkcja=lambda x: x == 'Tak' or x == 'metallic', value_none=False)
pol_ang_con(df_all, 'param_akryl-(niemetalizowany)', 'param_acrylic', 'param_akryl-(niemetalizowany)_new', funkcja=lambda x: x == 'Tak' or x == 'acrylic', value_none=False)
pol_ang_con(df_all, 'param_financing-option', 'param_możliwość-finansowania', 'param_możliwość-finansowania_new', funkcja=bool_funk, value_none=False)
pol_ang_con(df_all, 'param_bezwypadkowy', 'param_no-accident', 'param_bezwypadkowy_new', funkcja=bool_funk, value_none=False)
pol_ang_con(df_all, 'param_uszkodzony', 'param_damaged', 'param_uszkodzony_new', funkcja=bool_funk, value_none=False)
pol_ang_con(df_all, 'param_perłowy', 'param_pearl', 'param_perłowy_new', funkcja=bool_funk, value_none=False)
pol_ang_con(df_all, 'param_service-record', 'param_serwisowany-w-aso', 'param_serwisowany-w-aso_new', funkcja=bool_funk, value_none=False)
pol_ang_con(df_all, 'param_particle-filter', 'param_filtr-cząstek-stałych', 'param_filtr-cząstek-stałych_new', funkcja=bool_funk, value_none=False)
pol_ang_con(df_all, 'param_zarejestrowany-w-polsce', 'param_registered-in-poland', 'param_zarejestrowany-w-polsce_new', funkcja=bool_funk, value_none=False)
pol_ang_con(df_all, 'param_miesięczna-rata', 'param_monthly-payment-value', 'param_miesięczna-rata_new', funkcja=lambda x: (float_filter(x)), value_none=-1)
pol_ang_con(df_all, 'param_model-pojazdu', 'param_model', 'param_model-pojazdu_new', value_none='Other')
pol_ang_con(df_all, 'param_marka-pojazdu', 'param_make', 'param_marka-pojazdu_new', value_none='Other')
pol_ang_con(df_all, 'param_napęd', 'param_transmission', 'param_napęd_new')
pol_ang_con(df_all, 'param_skrzynia-biegów', 'param_gearbox', 'param_skrzynia-biegów_new')
pol_ang_con(df_all, 'param_kolor', 'param_color', 'param_kolor_new', funkcja=lambda x: x.lower())
pol_ang_con(df_all, 'param_emisja-co2', 'param_co2-emissions', 'param_emisja-co2_new', funkcja=lambda x: x.lower())
pol_ang_con(df_all, 'param_typ', 'param_body-type', 'param_typ_new', funkcja=lambda x: x.lower())
pol_ang_con(df_all, 'param_version', 'param_wersja', 'param_wersja_new', funkcja=lambda x: x.lower())
pol_ang_con(df_all, 'param_kod-silnika', 'param_engine-code', 'param_kod-silnika_new', funkcja=lambda x: x.lower())
pol_ang_con(df_all, 'param_pojemność-skokowa', 'param_engine-capacity', 'param_pojemność-skokowa_new', value_none='unknown')
pol_ang_con(df_all, 'param_rodzaj-paliwa', 'param_fuel-type', 'param_rodzaj-paliwa_new')
pol_ang_con(df_all, 'param_liczba-miejsc', 'param_nr-of-seats', 'param_liczba-miejsc_new', funkcja=lambda x: int(x), value_none=-1)
pol_ang_con(df_all, 'param_liczba-drzwi', 'param_door-count', 'param_liczba-drzwi_new', funkcja=lambda x: int(x), value_none=-1)
pol_ang_con(df_all, 'param_kraj-pochodzenia', 'param_country-of-origin', 'param_kraj-pochodzenia_new')
pol_ang_con(df_all, 'param_leasing', 'param_leasing-concession', 'param_leasing_new', funkcja=bool_funk, value_none=False)

In [ ]:
df_all['param_wartość-wykupu_new'] = [int(num_filter(x)) if x != None else -1 for x in df_all['param_wartość-wykupu']]
del df_all['param_wartość-wykupu']
df_all['param_liczba-pozostałych-rat_new'] = [int(num_filter(x)) if x != None else -1 for x in df_all['param_liczba-pozostałych-rat']]
del df_all['param_liczba-pozostałych-rat']
df_all['param_opłata-początkowa_new'] = [int(num_filter(x)) if x != None else -1 for x in df_all['param_opłata-początkowa']]
del df_all['param_opłata-początkowa']

In [ ]:
df_all['param_faktura-vat'] = df_all['param_faktura-vat'].map(bool_funk)
df_all['param_homologacja-ciężarowa'] = df_all['param_homologacja-ciężarowa'].map(bool_funk)
df_all['param_original-owner'] = df_all['param_original-owner'].map(bool_funk)
df_all['param_vat-marża'] = df_all['param_vat-marża'].map(bool_funk)
df_all['param_leasing_new'] = df_all['param_leasing_new'].map(bool_funk)
df_all['param_zarejestrowany-jako-zabytek'] = df_all['param_zarejestrowany-jako-zabytek'].map(bool_funk)
df_all['param_pierwszy-właściciel'] = df_all['param_pierwszy-właściciel'].map(bool_funk)
df_all['param_vat-discount'] = df_all['param_vat-discount'].map(bool_funk)
df_all['param_kierownica-po-prawej-(anglik)'] = df_all['param_kierownica-po-prawej-(anglik)'].map(bool_funk)
df_all['param_vat-free'] = df_all['param_vat-free'].map(bool_funk)
df_all['param_tuning'] = df_all['param_tuning'].map(bool_funk)
df_all['param_matowy'] = df_all['param_matowy'].map(bool_funk)

## Rejestracja to_datetime

In [ ]:
pol_ang_con(df_all, 'param_pierwsza-rejestracja', 'param_first-registration', 'param_pierwsza-rejestracja_new', funkcja=lambda x: str(x.lower()))

df_all['param_pierwsza-rejestracja_new'] = df_all['param_pierwsza-rejestracja_new'].map(lambda x: przyk(x))
df_all['param_pierwsza-rejestracja_new'] = pd.to_datetime(df_all['param_pierwsza-rejestracja_new'])

df_all['param_pierwsza-rejestracja_year'] = df_all['param_pierwsza-rejestracja_new'].dt.year
df_all['param_pierwsza-rejestracja_year'].fillna(-1)

df_all['param_pierwsza-rejestracja_month'] = df_all['param_pierwsza-rejestracja_new'].dt.month
df_all['param_pierwsza-rejestracja_month'] =df_all['param_pierwsza-rejestracja_month'].fillna(-1)

del df_all['param_pierwsza-rejestracja_new']

## Marka pojazdu

In [ ]:
def marka_from_breadcrumb(row):
  if (row["param_rok-produkcji_new"] == None): 
    marka = row["breadcrumb"][1]
    return marka
  else:
    return row['param_marka-pojazdu_new']

df_all['param_marka-pojazdu_newest'] = df_all.apply(marka_from_breadcrumb, axis=1)

def model_from_breadcrumb(row):
  if (row["param_rok-produkcji_new"] == None): 
    model = row["breadcrumb"][1]
    return model
  else:
    return row['param_model-pojazdu_new']

df_all['param_model-pojazdu_newest'] = df_all.apply(model_from_breadcrumb, axis=1)


del df_all['param_model-pojazdu_new']
del df_all['param_marka-pojazdu_new']

In [ ]:
marki = 'param_marka-pojazdu_newest'
modele = 'param_model-pojazdu_newest'
df_all['param_model_marka-pojazdu'] = [marka + ' ' + model if marka != None and marka != 'Other' and model != None and model != 'Other' else 'Other' for marka, model in zip(df_all[marki], df_all[modele])]
car_list = set(df_all['param_model_marka-pojazdu'])

In [ ]:
train = df_all[ ~df_all.price_value.isnull() ].copy()
test = df_all[ df_all.price_value.isnull() ].copy()

In [ ]:
car_list_priceing = {}
for car in car_list:
  rows = train[train['param_model_marka-pojazdu'] == car]
  car_list_priceing[car] = (len(rows), round(np.mean(rows['price_value']), 2), np.median(rows['price_value']))

price_value_p0 = np.percentile(train['price_value'], 15)
price_value_p1 = np.percentile(train['price_value'], 40)
price_value_p2 = np.percentile(train['price_value'], 75)
price_value_p3 = np.percentile(train['price_value'], 95)

very_cheap = []
cheap = []
normal = []
expensive = []
very_expensive = []
for car, (len_rows, mean, median) in car_list_priceing.items():
   if len_rows < 5 and car != 'Other':
     if median < price_value_p0:
       very_cheap.append(car)
       train.loc[train['param_model_marka-pojazdu'] == car, 'param_model_marka-pojazdu'] = 'Other Very Cheap'
     elif median < price_value_p1:
       cheap.append(car)
       train.loc[train['param_model_marka-pojazdu'] == car, 'param_model_marka-pojazdu'] = 'Other Cheap'
     elif median < price_value_p2:
       normal.append(car)
       train.loc[train['param_model_marka-pojazdu'] == car, 'param_model_marka-pojazdu'] = 'Other Normal'
     elif median < price_value_p3:
       expensive.append(car)
       train.loc[train['param_model_marka-pojazdu'] == car, 'param_model_marka-pojazdu'] = 'Other Expensive'
     else:
       very_expensive.append(car)
       train.loc[train['param_model_marka-pojazdu'] == car, 'param_model_marka-pojazdu'] = 'Other Very Expensive'


In [ ]:
train['param_model_marka-pojazdu_cat'] = train['param_model_marka-pojazdu'].factorize()[0]

In [ ]:
car_list_priceing = {}
for car in car_list:
  rows = train[train['param_model_marka-pojazdu'] == car]
  car_list_priceing[car] = (len(rows), round(np.mean(rows['price_value']), 2), np.median(rows['price_value']))

very_cheap = []
cheap = []
normal = []
expensive = []
very_expensive = []
for car, (len_rows, mean, median) in car_list_priceing.items():
   if len_rows > 0 and car != 'Other':
     if median < price_value_p0:
       very_cheap.append(car)
       train.loc[train['param_model_marka-pojazdu'] == car, 'param_model_marka-pojazdu'] = 'Very Cheap'
     elif median < price_value_p1:
       cheap.append(car)
       train.loc[train['param_model_marka-pojazdu'] == car, 'param_model_marka-pojazdu'] = 'Cheap'
     elif median < price_value_p2:
       normal.append(car)
       train.loc[train['param_model_marka-pojazdu'] == car, 'param_model_marka-pojazdu'] = 'Normal'
     elif median < price_value_p3:
       expensive.append(car)
       train.loc[train['param_model_marka-pojazdu'] == car, 'param_model_marka-pojazdu'] = 'Expensive'
     else:
       very_expensive.append(car)
       train.loc[train['param_model_marka-pojazdu'] == car, 'param_model_marka-pojazdu'] = 'Very Expensive'

train['param_car_categories_cat'] = train['param_model_marka-pojazdu'].factorize()[0]
del train['param_model_marka-pojazdu']

## Factorize

In [ ]:
obj_feats = list(df_all.select_dtypes(np.object).columns)
obj_feats = [x for x in obj_feats if x.startswith('param') or x.startswith('seller')]


for feat in obj_feats:
    df_all["{}_cat".format(feat)] = df_all[feat].factorize()[0]
    


In [ ]:
black_list = ['param_vin', 'param_kategoria', 'car_id', 'price_details', 'price_value', 'price_values']
feats = df_all.select_dtypes(include=[np.number, np.float, np.bool]).columns
feats = [x for x in feats if x not in black_list]

## Model

In [ ]:
train = df_all[ ~df_all.price_value.isnull() ].copy()
test = df_all[ df_all.price_value.isnull() ].copy()

In [ ]:
xgb_params = {
    'objective': 'reg:squarederror',
    'n_jobs': 4,
    'max_depth': 11, 
    'n_estimators': 400,
    'learning_rate': 0.055,
    'random_state': 9700,
    'colsample_bytree': 0.81,
    'min_child_weight': 1,
    'reg_alpha': 1.0835,
    'reg_lambda': 0.94,
    'subsample': 0.89
}

shift = 10000
X = train[ feats ].values
y_train = train['price_value'].values
y_train_log = np.log(y_train + shift)

cv = KFold(n_splits=3, shuffle=True, random_state=0)
model = xgb.XGBRegressor(**xgb_params)
scores = []


In [ ]:
for train_idx, test_idx in cv.split(X):
    model.fit(X[train_idx], y_train_log[train_idx])
    y_log_pred = model.predict(X[test_idx])
    y_log_pred[ y_log_pred < 0 ] = 1e-6
        
    y_pred = np.exp( y_log_pred ) - shift
    
    global_min = train.price_value.min()
    y_pred[ y_pred < global_min] = global_min

    score = mae(y_train[test_idx], y_pred)
    scores.append(score)
    print(np.mean(scores), np.std(scores))

In [ ]:
skplt.estimators.plot_feature_importances(model, feature_names=feats, x_tick_rotation=90, figsize=(15, 5));

In [ ]:
skplt.estimators.plot_learning_curve(model, X, y_train, figsize=(15, 5), cv=3, scoring='neg_mean_absolute_error');

## Sprawdzenie gdzie model popełnia największe błędy

In [ ]:
price_value_pred = {}
cv = KFold(n_splits=2)

for train_idx, test_idx in tqdm(cv.split(X)):
    X_train, y_log_train = X[train_idx], y_train_log[train_idx]
    X_test, y_log_test   = X[test_idx], y_train_log[test_idx]
    
    model = xgb.XGBRegressor(**xgb_params)
    model.fit(X_train, y_log_train)

    y_log_pred = model.predict(X[test_idx])
    y_log_pred[ y_log_pred < 0 ] = 1e-6
        
    y_pred = np.exp( y_log_pred ) - shift
    
    global_min = train.price_value.min()
    y_pred[ y_pred < global_min] = global_min

    price_value_pred.update( dict(zip(list(test_idx), list(y_pred))) )
    
train['price_value_pred'] = train.index.map(price_value_pred)

In [ ]:
train['pred_diff'] = train['price_value'] - train['price_value_pred']
train['pred_diff_abs'] = train['pred_diff'].abs()

In [ ]:
train[ ['price_value', 'price_value_pred', 'pred_diff_abs'] ].sort_values(by='pred_diff_abs', ascending=False)